In [1]:
# Install the necessary packages
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q
!pip install pypdf -qU


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:00
 

 Initialize Hugging Face LLM

In [43]:
import os
from langchain_community.llms import HuggingFaceHub

# Set the Hugging Face API token as an environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "########"

# Initialize the Hugging Face LLM
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500},
    # The token will be automatically picked up from the environment variable
)


Initialize the Embedding Model

In [44]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)


Define the Output Parser

In [45]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

Load the PDF Document

In [46]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("sp1.pdf")  # Replace with your file path
docs = loader.load()

# Check the number of documents loaded
print(f"Number of documents loaded: {len(docs)}")


Number of documents loaded: 7


In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

# Check the number of splits
print(f"Number of chunks created: {len(splits)}")


Number of chunks created: 34


Create Vector Store and Retriever

In [48]:
from langchain.vectorstores import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()


Define Prompt Template

In [49]:
from langchain.prompts import ChatPromptTemplate

# Define the prompt template
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)


In [50]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'), additional_kwargs={})])

Chain Retriever, Prompt, and LLM

In [51]:
from langchain.schema.runnable import RunnablePassthrough

# Chain the retriever, prompt, and LLM
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)


Test the RAG pipeline with an example question

In [52]:

response = chain.invoke("what is the  Monte Carlo methord?")
print(response)


Human: 
Answer this question using the provided context only.

what is the  Monte Carlo methord?

Context:
[Document(metadata={'page': 2, 'source': 'sp1.pdf'}, page_content='Monte Carlo integration\n• Monte Carlo integration is a technique for numerical integration using random numbers.\n• It is a particular Monte Carlo method that numerically computes a definite integral.\n• Monte Carlo randomly chooses points at which the integrand is evaluated.\n• This method is particularly useful for higher-dimensional integrals (a multiple integral is a definite'), Document(metadata={'page': 2, 'source': 'sp1.pdf'}, page_content='Monte Carlo integration\n• Monte Carlo integration is a technique for numerical integration using random numbers.\n• It is a particular Monte Carlo method that numerically computes a definite integral.\n• Monte Carlo randomly chooses points at which the integrand is evaluated.\n• This method is particularly useful for higher-dimensional integrals (a multiple integral is 